In [29]:
# !pip install deep_translator
# !pip install langdetect
# !pip install langid

In [30]:
import pandas as pd
import langid
import requests
import re
import pickle


In [31]:

## import of the data
users_data = pd.read_csv("C:/Users/ASUS/anaconda3/data_scentics/data/external/users.csv").convert_dtypes()
ratings_data = pd.read_csv("C:/Users/ASUS/anaconda3/data_scentics/data/external/ratings.csv").convert_dtypes()
books_data = pd.read_csv("C:/Users/ASUS/anaconda3/data_scentics/data/external/books.csv", low_memory=False)

In [32]:
# put the title and author all to upper case
books_data['Book-Title'] = books_data['Book-Title'].str.upper()
books_data['Book-Author'] = books_data['Book-Author'].str.upper()
books_data['Book-Title'] = books_data['Book-Title'].astype(str)
books_data['Publisher'] = books_data['Publisher'].str.upper()
books_data = books_data[['ISBN', 'Book-Author', 'Book-Title','Year-Of-Publication','Publisher']]

pattern = r'[():]'
books_data['Book-Title'] = books_data['Book-Title'].apply(lambda x:re.sub(pattern, '', x))
books_data['Language'] = None

In [33]:
# Checking  for  null value Book-Author
books_data[books_data['Book-Author'].isna()]


,ISBN,Book-Author,Book-Title,Year-Of-Publication,Publisher,Language
118033,0751352497,NaN,A+ QUIZ MASTERS01 EARTH,1999,DORLING KINDERSLEY,None
187689,9627982032,NaN,THE CREDIT SUISSE GUIDE TO MANAGING YOUR PERSO...,1995,EDINBURGH FINANCIAL PUBLISHING,None


In [34]:
# updating the null value
books_data.loc[187689, 'Book-Author'] = str.upper('Larissa Anne Downes')
books_data.loc[118033, 'Book-Author'] = str.upper('Neznamy')

In [35]:
# Checking  for  null value Book-Title
books_data[books_data['Book-Title'].isna()]


,ISBN,Book-Author,Book-Title,Year-Of-Publication,Publisher,Language


In [36]:
# Checking  for  null value Year-Of-Publication
books_data[books_data['Year-Of-Publication'].isna()]


,ISBN,Book-Author,Book-Title,Year-Of-Publication,Publisher,Language


In [37]:
# Checking  for  null value Publisher
books_data[books_data['Publisher'].isna()]

,ISBN,Book-Author,Book-Title,Year-Of-Publication,Publisher,Language
128890,193169656X,ELAINE CORVIDAE,TYRANT MOON,2002,NaN,None
129037,1931696993,LINNEA SINCLAIR,FINDERS KEEPERS,2001,NaN,None


In [38]:
# updating the null values
books_data.loc[128890, 'Publisher'] = str.upper('NovelBooks, Inc.')
books_data.loc[129037, 'Publisher'] = str.upper('CreateSpace Independent Publishing Platform')

In [39]:
# overview about values in Year-Of-Publication
books_data['Year-Of-Publication'].unique()

array(['2002', '2001', '1991', '1999', '2000', '1993', '1996', '1988',
       '2004', '1998', '1994', '2003', '1997', '1983', '1979', '1995',
       '1982', '1985', '1992', '1986', '1978', '1980', '1952', '1987',
       '1990', '1981', '1989', '1984', '0', '1968', '1961', '1958',
       '1974', '1976', '1971', '1977', '1975', '1965', '1941', '1970',
       '1962', '1973', '1972', '1960', '1966', '1920', '1956', '1959',
       '1953', '1951', '1942', '1963', '1964', '1969', '1954', '1950',
       '1967', '2005', '1957', '1940', '1937', '1955', '1946', '1936',
       '1930', '2011', '1925', '1948', '1943', '1947', '1945', '1923',
       '2020', '1939', '1926', '1938', '2030', '1911', '1904', '1949',
       '1932', '1928', '1929', '1927', '1931', '1914', '2050', '1934',
       '1910', '1933', '1902', '1924', '1921', '1900', '2038', '2026',
       '1944', '1917', '1901', '2010', '1908', '1906', '1935', '1806',
       '2021', '2012', '2006', 'DK Publishing Inc', 'Gallimard', '1909',
       

In [40]:
# info about the books with wrong Year-Of-Publication
books_data.loc[books_data['Year-Of-Publication'].isin(['DK Publishing Inc','Gallimard'])] 

,ISBN,Book-Author,Book-Title,Year-Of-Publication,Publisher,Language
209538,078946697X,2000,"DK READERS CREATING THE X-MEN, HOW IT ALL BEGA...",DK Publishing Inc,HTTP://IMAGES.AMAZON.COM/IMAGES/P/078946697X.0...,None
220731,2070426769,2003,"PEUPLE DU CIEL, SUIVI DE 'LES BERGERS\"";JEAN-M...",Gallimard,HTTP://IMAGES.AMAZON.COM/IMAGES/P/2070426769.0...,None
221678,0789466953,2000,"DK READERS CREATING THE X-MEN, HOW COMIC BOOKS...",DK Publishing Inc,HTTP://IMAGES.AMAZON.COM/IMAGES/P/0789466953.0...,None


In [41]:
# correcting the Year-Of-Publication
books_data.loc[209538,'Year-Of-Publication'] = 2000
books_data.loc[220731,'Year-Of-Publication'] = 2003
books_data.loc[221678,'Year-Of-Publication'] = 2000

In [42]:
#check of Year-Of-Publication
books_data['Year-Of-Publication'].unique()

array(['2002', '2001', '1991', '1999', '2000', '1993', '1996', '1988',
       '2004', '1998', '1994', '2003', '1997', '1983', '1979', '1995',
       '1982', '1985', '1992', '1986', '1978', '1980', '1952', '1987',
       '1990', '1981', '1989', '1984', '0', '1968', '1961', '1958',
       '1974', '1976', '1971', '1977', '1975', '1965', '1941', '1970',
       '1962', '1973', '1972', '1960', '1966', '1920', '1956', '1959',
       '1953', '1951', '1942', '1963', '1964', '1969', '1954', '1950',
       '1967', '2005', '1957', '1940', '1937', '1955', '1946', '1936',
       '1930', '2011', '1925', '1948', '1943', '1947', '1945', '1923',
       '2020', '1939', '1926', '1938', '2030', '1911', '1904', '1949',
       '1932', '1928', '1929', '1927', '1931', '1914', '2050', '1934',
       '1910', '1933', '1902', '1924', '1921', '1900', '2038', '2026',
       '1944', '1917', '1901', '2010', '1908', '1906', '1935', '1806',
       '2021', '2012', '2006', 2000, 2003, '1909', '2008', '1378', '1919',
     

In [43]:
# convertingt the Year-Of-Publication to numeric
books_data['Year-Of-Publication'] = pd.to_numeric(books_data['Year-Of-Publication'], errors='coerce')

In [44]:
# find the Book-Author, which is only number
numeric_rows = books_data['Book-Author'].str.contains('^\d+$')
books_data[numeric_rows.fillna(False)]


,ISBN,Book-Author,Book-Title,Year-Of-Publication,Publisher,Language
209538,078946697X,2000,"DK READERS CREATING THE X-MEN, HOW IT ALL BEGA...",2000,HTTP://IMAGES.AMAZON.COM/IMAGES/P/078946697X.0...,None
220731,2070426769,2003,"PEUPLE DU CIEL, SUIVI DE 'LES BERGERS\"";JEAN-M...",2003,HTTP://IMAGES.AMAZON.COM/IMAGES/P/2070426769.0...,None
221678,0789466953,2000,"DK READERS CREATING THE X-MEN, HOW COMIC BOOKS...",2000,HTTP://IMAGES.AMAZON.COM/IMAGES/P/0789466953.0...,None


In [45]:
# update the Book-Author with the right values
books_data.loc[209538, 'Book-Author'] = str.upper('Teitelbaum, Michael')
books_data.loc[220731, 'Book-Author'] = str.upper('Le Clezio, Jean-Marie Gustave')
books_data.loc[221678, 'Book-Author'] = str.upper('Buckley Jr., James')

In [46]:
# checking the Book-Author with '. ' - problem with J.K.Rowling
books_data[books_data["Book-Author"].str.contains('. ')]

,ISBN,Book-Author,Book-Title,Year-Of-Publication,Publisher,Language
0,0195153448,MARK P. O. MORFORD,CLASSICAL MYTHOLOGY,2002,OXFORD UNIVERSITY PRESS,None
1,0002005018,RICHARD BRUCE WRIGHT,CLARA CALLAN,2001,HARPERFLAMINGO CANADA,None
2,0060973129,CARLO D'ESTE,DECISION IN NORMANDY,1991,HARPERPERENNIAL,None
3,0374157065,GINA BARI KOLATA,FLU THE STORY OF THE GREAT INFLUENZA PANDEMIC ...,1999,FARRAR STRAUS GIROUX,None
4,0393045218,E. J. W. BARBER,THE MUMMIES OF URUMCHI,1999,W. W. NORTON &AMP; COMPANY,None
...,...,...,...,...,...,...
271354,0449906736,ROBIN WRIGHT,FLASHPOINTS PROMISE AND PERIL IN A NEW WORLD,1993,BALLANTINE BOOKS,None
271355,0440400988,PAULA DANZIGER,THERE'S A BAT IN BUNK FIVE,1988,RANDOM HOUSE CHILDRENS PUB (MM),None
271356,0525447644,TERI SLOAT,FROM ONE TO ONE HUNDRED,1991,DUTTON BOOKS,None
271357,006008667X,CHRISTINE WICKER,LILY DALE THE TRUE STORY OF THE TOWN THAT TAL...,2004,HARPERSANFRANCISCO,None


In [47]:
# getting rid of the empty space
books_data['Book-Author'] = books_data['Book-Author'].apply(lambda x:re.sub('. ', '.', x))

In [49]:
#filling the Language column
books_data['Language'] = books_data['Book-Title'].apply(lambda x: langid.classify(x)[0])
books_data['Language'] = books_data['Language'].apply(lambda x: str.upper(x))

KeyboardInterrupt: 

In [50]:
# checking duplicates
books_data[books_data.duplicated()]

,ISBN,Book-Author,Book-Title,Year-Of-Publication,Publisher,Language


In [51]:
# checking null values
ratings_data[ratings_data['User-ID'].isna()]

,User-ID,ISBN,Book-Rating


In [52]:
# checking null values
ratings_data[ratings_data['ISBN'].isna()]

,User-ID,ISBN,Book-Rating


In [53]:
# checking null values
ratings_data[ratings_data['Book-Rating'].isna()]

,User-ID,ISBN,Book-Rating


In [54]:
# checking for duplicates
ratings_data[ratings_data.duplicated()]

,User-ID,ISBN,Book-Rating


In [55]:
# checking null values
users_data[users_data['User-ID'].isna()]

,User-ID,Location,Age


In [56]:
# checking null values
users_data[users_data['Location'].isna()]

,User-ID,Location,Age


In [57]:
# checking null values
users_data[users_data['Age'].isna()]

,User-ID,Location,Age
0,1,"nyc, new york, usa",<NA>
2,3,"moscow, yukon territory, russia",<NA>
4,5,"farnborough, hants, united kingdom",<NA>
6,7,"washington, dc, usa",<NA>
7,8,"timmins, ontario, canada",<NA>
...,...,...,...
278849,278850,"sergnano, lombardia, italy",<NA>
278853,278854,"portland, oregon, usa",<NA>
278855,278856,"brampton, ontario, canada",<NA>
278856,278857,"knoxville, tennessee, usa",<NA>


In [58]:
# filling null values
users_data['Age'].fillna(0, inplace=True)
users_data['Location'].fillna('', inplace=True)

In [60]:
# saving the data in interim folder

with open('C:/Users/ASUS/anaconda3/data_scentics/data/interim/books_data.pkl', 'wb') as f:
    pickle.dump(books_data, f)

with open('C:/Users/ASUS/anaconda3/data_scentics/data/interim/ratings_data.pkl', 'wb') as f:
    pickle.dump(ratings_data, f)

with open('C:/Users/ASUS/anaconda3/data_scentics/data/interim/users_data.pkl', 'wb') as f:
    pickle.dump(users_data, f)

